# Vitessce integration

For more comprehensive integration tests, see: https://github.com/laminlabs/lamin-spatial

In [ ]:
!lamin login testuser1
!lamin init --storage "s3://lamindb-ci/test-vitessce"

In [ ]:
import lamindb as ln
import pytest
from vitessce import (
    VitessceConfig,
    AnnDataWrapper,
)

Set up test data:

In [ ]:
pbmc68k = ln.core.datasets.anndata_pbmc68k_reduced()[:100, :200].copy()
zarr_filepath = "my_test.zarr"
# write the anndata to a local zarr path
pbmc68k.write_zarr(zarr_filepath)
# create an artifact from the path
dataset_artifact = ln.Artifact(zarr_filepath, description="Test dataset").save()
# this is the where the zarr folder is located on a public S3 bucket
dataset_artifact.path.to_url()

Create a `VitessceConfig` object: 

In [ ]:
vc = VitessceConfig(schema_version="1.0.15")
vc.add_dataset(name="test1").add_object(
    AnnDataWrapper(
        adata_artifact=dataset_artifact,
        obs_embedding_paths=["obsm/X_umap"],
    ),
)
vc.to_dict()

In [ ]:
vitessce_config_artifact = ln.integrations.save_vitessce_config(
    vc, description="View testdata in Vitessce"
)

In [ ]:
# different equivalent ways of testing that the action is attached
assert dataset_artifact._actions.get() == vitessce_config_artifact
assert vitessce_config_artifact._action_targets.get() == dataset_artifact
assert vitessce_config_artifact._actions.first() is None
assert ln.Artifact.get(_actions=vitessce_config_artifact) == dataset_artifact

In [ ]:
dataset_artifact.delete(permanent=True)
vitessce_config_artifact.delete(permanent=True)

Test validation within `save_vitessce_config`:

In [ ]:
# pass an artifact URL instead of the artifact object itself
vc = VitessceConfig(schema_version="1.0.15")
with pytest.raises(AttributeError) as error:
    vc.add_dataset(name="test1").add_object(
        AnnDataWrapper(
            adata_artifact=dataset_artifact.path.to_url(),
            obs_embedding_paths=["obsm/X_umap"],
        ),
    )
print(error.exconly())
assert error.exconly().startswith(
    "AttributeError: 'str' object has no attribute 'path'"
)

In [ ]:
!rm -rf test-vitessce
!lamin delete --force test-vitessce